# TensorflowTutorialMNIST_CNN


In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np


In [139]:
def cnn_model_fn(features, labels, mode):
    tf.set_random_seed(1234)
    img_input = tf.reshape(features["train_X"], [-1,28,28,1])
    
    # Input layer - here as a conv2d layer #1
    conv1 = tf.layers.conv2d(inputs=img_input, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
    
    # Dropout layer #1
    dropout1 = tf.layers.dropout(inputs=conv1, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))
    
    # Conv2d layer #2
    conv2 = tf.layers.conv2d(inputs=dropout1, filters=32, kernel_size=[5,5], padding="same", activation=tf.nn.relu)
      
    # Dropout layer #2
    dropout2 = tf.layers.dropout(inputs=conv2, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN, seed=(1234))

    flat_layer = tf.reshape(dropout2, [-1, 28 * 28 * 32])
    
    # Dense layer
    dense = tf.layers.dense(inputs=flat_layer, units=1024, activation=tf.nn.relu)
    
    # Logits layer
    logits = tf.layers.dense(inputs=dense, units=10)
    
    #print(logits)
    
    # Generate predictions
    #tf.argmax(input=logits)
    #print(logits)
    # Apply the softmax activation function
    #tf.nn.softmax(logits, name="softmax_tensor")
    
    # Compile the predictions in a dict and return EstimatorSpec as object
    predictions = {
        'classes': tf.argmax(input=logits, axis=1), 
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate loss
    print(labels)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Training sgd
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation - accuracy metrics
    eval_metrice = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrice)

In [140]:
from tensorflow.keras.datasets.mnist import load_data
(train_X, train_y),(test_X, test_y) = load_data()

train_X = train_X/np.float(255)
test_X = test_X/np.float(255)



In [141]:
# Estimator
clf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="minist_convent_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'minist_convent_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002709D4BEC18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [142]:
# Set up logging for predictions - softmax_tensor from above
tensor_to_log = {'probabilities': 'softmax_tensor'}

# Log after every 50 steps 
logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=50)

In [144]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"train_X": train_X},
    y=tf.dtypes.cast(train_y, tf.int64),
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
clf.train(
    input_fn=train_input_fn,
    steps=10,
    hooks=[logging_hook])

TypeError: unhashable type: 'Dimension'

In [37]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

NameError: name 'mnist_classifier' is not defined

In [38]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

NameError: name 'eval_data' is not defined